In [3]:
import pandas as pd
import plotly.graph_objects as go

# Load data
TFEX_S50 = pd.read_excel("S50_TFEX.xlsx")
TFEX_S50['Timestamp'] = pd.to_datetime(TFEX_S50['Timestamp'])
TFEX_S50.set_index('Timestamp', inplace=True)

set50 = pd.read_csv('SET50.csv')
set50['Timestamp'] = pd.to_datetime(set50['Date'])
set50.set_index('Timestamp', inplace=True)
set50.drop(columns=['Date'], inplace=True)
set50['Close'] = pd.to_numeric(set50['Close'].str.replace(',', ''), errors='coerce')

# Parameters
window = 30
cols = TFEX_S50.columns[:1]

for col in cols:
    print(f"Processing security: {col}")

    # Join and process data
    data = TFEX_S50[[col]].join(set50, how='inner').dropna()
    data[col] = pd.to_numeric(data[col], errors='coerce')
    data['SET50'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Gap'] = data[col] - data['SET50']
    data['Rolling Mean'] = data['Gap'].rolling(window=window).mean()
    data['Rolling Std'] = data['Gap'].rolling(window=window).std()
    data['Z-score'] = (data['Gap'] - data['Rolling Mean']) / data['Rolling Std']

    # Create traces for prices
    trace1 = go.Scatter(x=data.index, y=data[col], mode='lines', name=f'{col} Price')
    trace2 = go.Scatter(x=data.index, y=data['SET50'], mode='lines', name='SET50 Price')
    trace3 = go.Scatter(x=data.index, y=data['SET50'] - data[col], mode='lines', name='Price Difference')
    trace4 = go.Scatter(x=data.index, y=data['Z-score'], mode='lines', name='Z-Score')

    # Create a figure with price traces
    fig = go.Figure()
    # fig.add_trace(trace1)
    # fig.add_trace(trace2)
    # fig.add_trace(trace3)
    fig.add_trace(trace4)
    
    # Update layout
    fig.update_layout(
        title=f'Price Comparison: {col} vs SET50',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Legend',
        template='plotly_dark'
    )
    fig.show()


Processing security: S50H0^2
